Elastic potential energy of a continuum described in https://en.wikipedia.org/wiki/Elastic_energy#Continuum_systems

Try to figure out how to read Einstein notation.

In [1]:
import numpy as np

$(\delta_{ij}\delta_{kl})\epsilon_{ij}\epsilon_{kl}$ - trace squared?

c is 1 when:
 - i==j==k==l -> sum of square of diagonal elements
 - i==j and k==l -> twice the sum of product of diagonal elements with different indices

In [4]:
c = np.einsum('ij,kl->ijkl', np.eye(3), np.eye(3))
e = [[1,2,3],[4,5,6],[7,8,9]]
assert np.einsum("ijkl,ij,kl->", c, e, e) == 225
e = [[1,0,0],[0,2,0],[0,0,3]]
assert np.einsum("ijkl,ij,kl->", c, e, e) == 36
e = np.array([[1,100,99],[-99,2,100],[-100,-9999.99,3]])
assert np.einsum("ijkl,ij,kl->", c, e, e) == 36
assert np.einsum("ijkl,ij,kl->", c, -e, -e) == 36

$(\delta_{ik}\delta_{jl})\epsilon_{ij}\epsilon_{kl}$ - sum of squares of elements?

c is 1 when:
 - i==k and j==l

In [5]:
c = np.einsum('ik,jl->ijkl', np.eye(3), np.eye(3))
e = np.array([[1,2,3],[4,5,6],[7,8,9]])
assert np.einsum("ijkl,ij,kl->", c, e, e) == np.sum(e*e)
e = np.array([[1,100,99],[-99,2,100],[-100,-9999.99,3]])
assert np.einsum("ijkl,ij,kl->", c, e, e) == np.sum(e*e)

$(\delta_{il}\delta_{jk})\epsilon_{ij}\epsilon_{kl}$ - sum of products of e[i][j] and e[j][i]?

c is 1 when:
 - i==l and j==k

In [7]:
c = np.einsum('il,jk->ijkl', np.eye(3), np.eye(3))
e = np.array([[1,2,3],[4,5,6],[7,8,9]])
assert np.einsum("ijkl,ij,kl->", c, e, e) == np.sum(e*e.T)
e = np.array([[1,100,99],[-99,2,100],[-100,-9999.99,3]])
assert np.einsum("ijkl,ij,kl->", c, e, e) == np.sum(e*e.T)

Elastic potential energy due to deformation:

$U = \frac{1}{2}V_0\left(\lambda\delta_{ij}\delta_{kl}+\mu(\delta_{ik}\delta_{jl}+\delta_{il}\delta{jk})\right)\epsilon_{ij}\epsilon_{kl}$

$U = \frac{1}{2}V_0\left(\lambda\sum_{ij}\epsilon_{ii}\epsilon_{jj}+\mu\left(\sum_{ij}\epsilon_{ij}^2+\sum_{ij}\epsilon_{ij}\epsilon_{ji}\right)\right)$

$\lambda = K-\frac{2}{3}G,\ \mu = G$ where $K$ is the bulk modulus and $G$ is the shear modulus. It requires investigation whether this formula is 3D-only or works in any number of dimensions.

https://ocw.mit.edu/courses/2-002-mechanics-and-materials-ii-spring-2004/f7b561fbe8762ff73da9236e6c0fff73_lec7.pdf

Deformation gradient: $\frac{\partial u}{\partial x}$

$x_1 = x_0 + u \implies \Delta x = \frac{\partial u}{\partial x} x_0 \implies \frac{\partial u}{\partial x} = \Delta x x_0^{-1}$

 - $\mathbf{x} = \begin{bmatrix}x_1 & x_2\end{bmatrix}$
 - $\frac{\partial u}{\partial x} = \begin{bmatrix}\frac{\partial u}{\partial x_x}&\frac{\partial u}{\partial x_y}\end{bmatrix}$

For a given triangle/tetrahedron with one vertex at the origin, $\frac{\partial u}{\partial x}$ can be solved from the initial and deformed coordinates of other vertices. The inverse of $x_0$ can be precomputed.

For a sheet of cloth with a R2 parameter space and a R3 coordinate space:

Define a $3\times2$ matrix $T$ that transforms the R2 vector $x$ to R3.

$Tx_1=Tx_0+Tu \implies T\Delta x=T\frac{\partial u}{\partial x}x_0 \implies T\frac{\partial u}{\partial x}=T\Delta xx_0^{-1}$ ??

(Seems like $T\frac{\partial u}{\partial x}$ is the same as $w$ in the Baraff "[Large Steps in Cloth Simulation](https://www.cs.cmu.edu/~baraff/papers/sig98.pdf)" paper.)

$T$ depends on $x$, which introduces nonlinearity to the equation.

Strain tensor: $\epsilon = \frac{1}{2}\left(\frac{\partial u}{\partial x}+{\frac{\partial u}{\partial x}}^T\right)$

$U = \frac{1}{2}V_0\left(\lambda\sum_{ij}\epsilon_{ii}\epsilon_{jj}+\mu\left(\sum_{ij}\epsilon_{ij}^2+\sum_{ij}\epsilon_{ij}\epsilon_{ji}\right)\right)$

$\frac{\partial U}{\partial\epsilon_{ii}}=\frac{1}{2}V_0\left(\lambda\left(\epsilon_{ii}+\sum_{j}\epsilon_{jj}\right)+\mu\cdot4\epsilon_{ii}\right)$

$\frac{\partial U}{\partial\epsilon_{ij,i\ne j}}=\frac{1}{2}V_0\cdot\mu\left(2\epsilon_{ij}+\epsilon_{ji}\right)$